# Build the ETL Pipeline

In [2]:
import pandas as pd
from sqlalchemy import create_engine

In [3]:
user = 'root'
password = 'Sql_08sds'
host = 'localhost'
database = 'testdb'

# Create SQLAlchemy engine & Check connection

In [5]:
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{database}")

print(" Connected to MySQL successfully!")

 Connected to MySQL successfully!


# Extract customers from MySQL & Extract complaints from CSV

In [7]:
customers = pd.read_sql("SELECT * FROM customers", engine)

complaints = pd.read_csv("complaints.csv")
print("Rows extracted -> Customers:", len(customers), "Complaints:", len(complaints))
display(customers.head(), complaints.head())

Rows extracted -> Customers: 5 Complaints: 5


,customer_id,name,plan_type,join_date,region
0,1001,Asha Mehta,Prepaid,2023-05-12,Delhi
1,1002,Ravi Kumar,Postpaid,2022-12-20,Mumbai
2,1003,Sneha Rao,Prepaid,2023-01-18,Chennai
3,1004,Manoj Singh,Postpaid,2021-11-05,Delhi
4,1005,Divya Jain,Prepaid,2023-03-28,Kolkata


,complaint_id,customer_id,category,description,created_at,status
0,CMP-001,1002,Billing,Charged extra for data usage,2025/09/25 10:45,Open
1,CMP-002,1004,Network,Frequent call drops in Delhi,2025-09-25 09:30,Open
2,CMP-003,1005,Recharge,Recharge failed; amount deducted,25-09-2025 14:00,Closed
3,CMP-004,1002,Network,Slow 4G speed at night,2025-09-26 20:40,Open
4,CMP-005,1003,Support,No response to complaint,2025-09-26 11:10,Open


# Transform Data (Clean, Fix, and Merge)

In [8]:
# --- Standardize text ---

customers['region'] = customers['region'].str.title().str.strip()
complaints['status'] = complaints['status'].str.title().str.strip()
complaints['category'] = complaints['category'].str.title().str.strip()

# --- Parse and standardize dates ---
customers['join_date'] = pd.to_datetime(customers['join_date'], errors='coerce')
complaints['created_at'] = pd.to_datetime(complaints['created_at'], errors='coerce')

# Fill unparseable or missing dates
default_dt = pd.Timestamp('2025-09-25 00:00')
customers['join_date'] = customers['join_date'].fillna(default_dt)
complaints['created_at'] = complaints['created_at'].fillna(default_dt)

# --- Fix missing IDs or text ---
complaints['customer_id'] = complaints['customer_id'].fillna(-1).astype(int)
complaints['description'] = complaints['description'].fillna("No description provided")

# --- Merge ---
merged = customers.merge(complaints, on='customer_id', how='left')

# --- Post-merge fixes ---
merged['complaint_id'] = merged['complaint_id'].fillna("NO-COMPLAINT")
merged['category'] = merged['category'].fillna("No Complaint")
merged['status'] = merged['status'].fillna("Resolved")
merged['created_at'] = merged['created_at'].fillna(default_dt)

# Derived flag
merged['is_open'] = (merged['status'] == 'Open')
print(" Data transformed successfully!")
merged.head()

 Data transformed successfully!


,customer_id,name,plan_type,join_date,region,complaint_id,category,description,created_at,status,is_open
0,1001,Asha Mehta,Prepaid,2023-05-12,Delhi,NO-COMPLAINT,No Complaint,NaN,2025-09-25 00:00:00,Resolved,False
1,1002,Ravi Kumar,Postpaid,2022-12-20,Mumbai,CMP-001,Billing,Charged extra for data usage,2025-09-25 10:45:00,Open,True
2,1002,Ravi Kumar,Postpaid,2022-12-20,Mumbai,CMP-004,Network,Slow 4G speed at night,2025-09-25 00:00:00,Open,True
3,1003,Sneha Rao,Prepaid,2023-01-18,Chennai,CMP-005,Support,No response to complaint,2025-09-25 00:00:00,Open,True
4,1004,Manoj Singh,Postpaid,2021-11-05,Delhi,CMP-002,Network,Frequent call drops in Delhi,2025-09-25 00:00:00,Open,True


# Load (Save the Output)

In [9]:
merged.to_csv("etl_output.csv", index=False)
print(f" ETL pipeline complete! Created {len(merged)} records and saved etl_output.csv.")

 ETL pipeline complete! Created 6 records and saved etl_output.csv.


# Validation & Quick Analytics

In [13]:
print("\n\n\nComplaints per customer:")
print(merged.groupby(['customer_id','name']).complaint_id.count().reset_index(name='complaint_count'))
print("\n\n\nOpen vs Closed:")
print(merged['status'].value_counts())
print("\n\n\nComplaints by region & category:")
print(merged.groupby(['region','category']).complaint_id.count().reset_index(name='count'))




Complaints per customer:
   customer_id         name  complaint_count
0         1001   Asha Mehta                1
1         1002   Ravi Kumar                2
2         1003    Sneha Rao                1
3         1004  Manoj Singh                1
4         1005   Divya Jain                1



Open vs Closed:
status
Open        4
Resolved    1
Closed      1
Name: count, dtype: int64



Complaints by region & category:
    region      category  count
0  Chennai       Support      1
1    Delhi       Network      1
2    Delhi  No Complaint      1
3  Kolkata      Recharge      1
4   Mumbai       Billing      1
5   Mumbai       Network      1
